1. Bagging, passo a passo:
   - **Criação de subconjuntos de dados**: A partir do conjunto de dados original, são criados vários subconjuntos de dados.
   - **Treinamento de modelos**: Cada subconjunto é usado para treinar um modelo independente.
   - **Combinação dos resultados**: Após o treinamento, os resultados dos modelos são combinados. Para problemas de classificação, a combinação é feita por votação majoritária. Para problemas de regressão, usamos a média dos resultados.

2. Bagging é uma técnica que treina vários modelos em vez de apenas um só. isso parte do princípio de que uma coleção de modelos pode ser mais precisa do que um único modelo.

### Vamos implementar o Bagging manualmente, sem usar bibliotecas específicas para isso.


In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import plotly.express as px

#### Aqui vamos utilizar o conjunto de dados `digits` do `sklearn`, que é um conjunto de dados clássico para classificação de dígitos manuscritos.


In [2]:
digits = load_digits()

X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

modelo = DecisionTreeClassifier(random_state=4)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)
print(f'A precisão do modelo no conjunto de treino: {modelo.score(X_train, y_train)*100:.2f} %')
print(f'A precisão do modelo no conjunto de teste: {modelo.score(X_test, y_test)*100:.2f} %')

A precisão do modelo no conjunto de treino: 100.00 %
A precisão do modelo no conjunto de teste: 86.11 %


Como visto acima, fizemos uma arvore simples de decisão para comparação.

E abaixo vamos criar 3 modelos de bootstraping (bagging) para ver um pouco como isso vai funcionar na prática.

Os codigos abaixo fazem o seguinte:
- Criam subconjuntos de dados com reposição (bootstrap samples). São subconjuntos aleatórios do conjunto de treinamento original com a mesma quantidade de linhas podendo ter linhas repetidas.
- Treinam um modelo de árvore de decisão em cada subconjunto.
- Fazem previsões em cada modelo treinado.
- Combinam as previsões dos modelos usando votação majoritária para obter a previsão final do bagging.

por fim, para cada linha de cada modelo, contamos quantas vezes cada classe foi prevista e escolhemos a classe com mais votos como a previsão final do bagging.

In [3]:
df_bagging = pd.DataFrame()
random_indices_000 = np.random.randint(
    low=0,
    high=len(X_train),
    size=len(X_train)
)
random_indices_000

df_bootstrap_000 = pd.DataFrame(X_train[random_indices_000])

modelo_000 = DecisionTreeClassifier(random_state=4)
modelo_000.fit(df_bootstrap_000, y_train[random_indices_000])
y_pred_000 = modelo_000.predict(X_test)
df_bagging['modelo_000'] = y_pred_000
df_bagging.value_counts()

modelo_000
2             44
5             43
6             36
1             35
7             35
0             34
8             34
3             33
4             33
9             33
Name: count, dtype: int64

In [4]:
random_indices_001 = np.random.randint(
    low=0,
    high=len(X_train),
    size=len(X_train)
)
random_indices_001

df_bootstrap_001 = pd.DataFrame(X_train[random_indices_001])

modelo_001 = DecisionTreeClassifier(random_state=4)
modelo_001.fit(df_bootstrap_001, y_train[random_indices_001])
y_pred_001 = modelo_001.predict(X_test)
df_bagging['modelo_001'] = y_pred_001
df_bagging

,modelo_000,modelo_001
0,8,8
1,7,7
2,0,0
3,5,5
4,3,1
...,...,...
355,8,8
356,5,5
357,4,4
358,3,3


In [5]:
random_indices_002 = np.random.randint(
    low=0,
    high=len(X_train),
    size=len(X_train)
)
random_indices_002

df_bootstrap_002 = pd.DataFrame(X_train[random_indices_002])

modelo_002 = DecisionTreeClassifier(random_state=4)
modelo_002.fit(df_bootstrap_002, y_train[random_indices_002])
y_pred_002 = modelo_001.predict(X_test)
df_bagging['modelo_002'] = y_pred_002
df_bagging

,modelo_000,modelo_001,modelo_002
0,8,8,8
1,7,7,7
2,0,0,0
3,5,5,5
4,3,1,1
...,...,...,...
355,8,8,8
356,5,5,5
357,4,4,4
358,3,3,3


In [6]:
df_bagging['voto_maj'] = df_bagging.mode(axis=1)[0]

Como podemos ver abaixo, no desultado abaixo, temos uma coluna com o voto majoritário de cada linha, que é a previsão final do modelo de Bagging.

Mas vamos implementar e definir algumas funções para realizar o Bagging de maneira mais prática. 

In [7]:
df_bagging

,modelo_000,modelo_001,modelo_002,voto_maj
0,8,8,8,8
1,7,7,7,7
2,0,0,0,0
3,5,5,5,5
4,3,1,1,1
...,...,...,...,...
355,8,8,8,8
356,5,5,5,5
357,4,4,4,4
358,3,3,3,3


#### Com todos os dados necessários vamos definir as funções e implementar esse bagging usando 1000 modelos de árvore de decisão (Decision Tree Classifier) como modelos base.

Aqui temos as funções para facilitar nosso processo de Bagging:

In [8]:
def gerar_indices_bootstrap(base_treino):
    return np.random.randint(
        low=0,
        high=len(base_treino),
        size=len(base_treino)
    )

def gerar_modelo_arvore_decisao(X_treino, y_treino, indices_bootstrap):
    df_bootstrap = pd.DataFrame(X_treino[indices_bootstrap])
    modelo = DecisionTreeClassifier(random_state=4)
    modelo.fit(df_bootstrap, y_treino[indices_bootstrap])
    return modelo

def gerar_dataframe_previsoes(modelo, X_teste, nome_coluna):
    y_pred = modelo.predict(X_teste)
    return pd.Series(y_pred, name=nome_coluna)


Aqui nos fazemos a mágica acontecer:
- A função `bootstrap_sample` cria um subconjunto de dados com reposição.
- A função `train_base_model` treina um modelo de árvore de decisão em um subconjunto de dados.
- A função `bagging_predict` faz previsões combinadas usando votação majoritária a partir dos modelos treinados.

In [9]:
lista_previsoes = []

for i in range(1000):
    indices_bootstrap = gerar_indices_bootstrap(X_train)
    modelo_bag = gerar_modelo_arvore_decisao(X_train, y_train, indices_bootstrap)
    nome_coluna = f'modelo_{i:03}'
    previsoes_series = gerar_dataframe_previsoes(modelo_bag, X_test, nome_coluna)
    
    lista_previsoes.append(previsoes_series)

df_bagging = pd.concat(lista_previsoes, axis=1)
df_bagging['voto_maj'] = df_bagging.mode(axis=1)[0]
df_bagging

,modelo_000,modelo_001,modelo_002,modelo_003,modelo_004,modelo_005,modelo_006,modelo_007,modelo_008,modelo_009,...,modelo_991,modelo_992,modelo_993,modelo_994,modelo_995,modelo_996,modelo_997,modelo_998,modelo_999,voto_maj
0,8,0,7,7,4,7,4,8,0,4,...,7,8,4,8,8,4,1,4,4,4
1,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,3,3,3,2,3,3,6,2,3,8,...,5,3,3,6,2,3,5,2,7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2,8,8,8,8,8,8,8,8,8,...,8,2,8,2,8,8,8,8,8,8
356,5,5,5,5,5,5,5,5,5,5,...,5,7,5,5,5,5,5,5,5,5
357,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,7,4,4
358,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


Por fim temos um modelo de Bagging completo, que podemos usar para treinar e fazer previsões em novos dados.
Vamos relembrar quais foram os resultados do modelo de árvore de decisão simples:

In [10]:
print(f'A precisão do modelo de arvore simples no conjunto de treino: {modelo.score(X_train, y_train)*100:.2f} %')
print(f'A precisão do modelo de arvore simples no conjunto de teste: {modelo.score(X_test, y_test)*100:.2f} %')

A precisão do modelo de arvore simples no conjunto de treino: 100.00 %
A precisão do modelo de arvore simples no conjunto de teste: 86.11 %


No df_bagging temos as previsões de cada modelo e a coluna final com a previsão do Bagging.
Vamos fazer a avaliação do modelo de Bagging:

In [11]:
df_previsao = pd.DataFrame()
df_previsao['bagging'] = df_bagging['voto_maj'].copy()
df_previsao['y_teste'] = y_test
df_previsao['acerto'] = np.where(df_previsao['bagging'] == df_previsao['y_teste'], 1, 0)
df_previsao['acerto'].mean()
print(f'A precisão dos modelos usando bagging: {df_previsao["acerto"].mean()*100:.2f} %')


A precisão dos modelos usando bagging: 96.39 %


### O resultado final do Bagging é uma melhoria significativa na precisão do modelo em comparação com a árvore de decisão simples, demonstrando a eficácia do Bagging na redução do overfitting e na melhoria da generalização do modelo.

In [12]:
a = modelo.score(X_test, y_test)*100
b = df_previsao["acerto"].mean()*100
c = np.abs(b - a)
print(f'Acurácia Árvore Simples: {a:.2f}%\nAcurácia Bagging: {b:.2f}%\nGanho de acurácia: {c:.2f}%')

fig = px.bar(
    x=['Árvore de Decisão Simples', 'Bagging'],
    y=[a, b],
    title=f'Comparação de Modelos<br>Ganho de acurácia: {c:.2f}%',
    labels={'x': 'Modelo', 'y': 'Acurácia (%)'},
    template='plotly_dark'
)
fig.add_hline(
    y=a,
    line_dash='dot',
    line_color='red',
    annotation_text=f'Linha<br>Árvore Simples: <br>{a:.2f}%', # <br> para quebra de linha
    annotation_position='top left',
    annotation_font_color='white'
)
fig.update_traces(marker_color=['blue', 'orange'])
fig.update_layout(width=400, height=600)
fig.show()

Acurácia Árvore Simples: 86.11%
Acurácia Bagging: 96.39%
Ganho de acurácia: 10.28%
